In [10]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
from sklearn.metrics import f1_score
import random
from copy import deepcopy

In [11]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [12]:
selected_model = Supported_modles.MLP_classifier

In [13]:
clients, test_x, test_y = utils.set_data(selected_model, True, True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [14]:
f1_before = []
for client in clients:
    client.init_empty_model(0.01)
    client.train_model()
    score = client.test_model_f1(y_test=test_y, X_test=test_x)
    f1_before.append(score)
    print(score)

In [5]:
fedavg = Fedavg("global", 0.1, selected_model)

Waitiing for a Connection..


In [6]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True

for client in clients:
    client.init_empty_model(0.01)
    X_train, X_test, y_train, y_test = client.split_data(0.8)

    client.train_model(X_train,y_train)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.init_global_model(client.model)
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.
.
.


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [7]:
score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
print(score)

0.8764589663130282


In [8]:
number_of_rounds = 4
epochs = 10
max_score = 0
optimal_model = None

for _ in range(number_of_rounds):
    print(_, end=' ')

    applicable_clients = random.sample((clients), 5)#random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        print(f'.', end='')

        X_train, X_test, y_train, y_test = client.split_data(0.8)
       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        client.load_global_model(fedavg.model)
        client.train_local_agent(X_train, y_train, epochs, sample_weights)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    score = fedavg.test_model_f1(y_test=test_y, X_test=test_x)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

0 ....0.8764589663130282
1 .2 ...3 .

In [9]:
x = test_x
y_hat = optimal_model.predict(x)
score = f1_score(test_y,y_hat)
print(score)

0.8764589663130282


In [ ]:
i = 0
for client in clients:
    print(f'Difference {score-f1_before[i]}')
    i += 1

Difference 0.003780109046955027
Difference 0.004070293970867334
Difference 0.0037914011297307537
Difference 0.0038043310180778667
Difference 0.0038860041655082167
